In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from functools import reduce
import copy
%matplotlib inline

# Vertex data

In [47]:
years = map(str,range(1999,2018))
with open('EU_countries.json') as f:
    countries = json.load(f)['results']
countries_id = [c['id'] for c in countries]
countries_name = {c['id']:c['text'] for c in countries}

files = {y: glob.glob('../../data/comtrade/Export_data_yearly/AG2/{}/Export_{}_*_0.csv'.format(y,y)) for y in years}
files = {y: [f for f in files[y] if f.split('_')[-2] in countries_id] for y in files}

In [48]:
available_countries = {y : [f.split('_')[-2] for f in files[y]] for y in files}
persisting_countries = reduce(lambda x,y: set.intersection(set(x),set(y)),available_countries.values())
print (len(persisting_countries))

28


In [49]:
comtrade = pd.concat([pd.read_csv(f) for f in files['1999']],ignore_index=1)[['Year','Reporter Code','Reporter', 'Partner Code', 'Partner', 'Commodity Code', 'Commodity','Trade Value (US$)']]


In [50]:
commodity = comtrade[['Commodity Code','Commodity']].drop_duplicates('Commodity Code',keep='first')
commodity.columns = ['id','name']

In [51]:
dict_code_id = {'AT' : '40',
               'BE' : '56',
               'BG' : '100',
               'HR' : '191',
               'CY' : '196',
               'CZ' : '203',
               'DK' : '208',
               'EE' : '233',
               'FI' : '246',
               'FR' : '251',
               'DE' : '276',
               'EL' : '300',
               'HU' : '348',
               'IE' : '372',
               'IT' : '381',
               'LV' : '428',
               'LT' : '440',
               'LU' : '442',
               'MT' : '470',
               'NL' : '528',
               'PL' : '616',
               'PT' : '620',
               'RO' : '642',
               'SK' : '703',
               'SI' : '705',
               'SE' : '752',
               'ES' : '724',
               'UK' : '826'}

dict_id_code = dict(zip(dict_code_id.values(),dict_code_id.keys()))

countries_df = comtrade[['Reporter Code','Reporter']].drop_duplicates()
countries_df.columns = ['id','name']
countries_df.id = [dict_id_code[str(c)] for c in countries_df.id]
countries_df.index = range(len(countries_df))

In [52]:
sitc_names = {0 : 'Food and live animals',
             1 : 'Beverages and tobacco',
             2 : 'Crude materials, inedible, except fuels',
             3 : 'Mineral fuels, lubricants and related materials',
             4 : 'Animal and vegetable oils, fats and waxes',
             5 : 'Chemicals and related products, n.e.s.',
             6 : 'Manufactured goods',
             7 : 'Machinery and transport equipment',
             8 : 'Miscellaneous manufactured articles',
             9 : 'Commodities and transactions not classified elsewhere'}

In [53]:
complexity = ((pd.read_csv('../competitiveness/YearlyComplexityAG2W.csv')))
fitness = pd.read_csv('../competitiveness/YearlyFitnessAG2W.csv')

In [54]:
hs_to_sitc = pd.read_csv('../../data/comtrade/HStoSITC.csv',sep='\t',dtype={'From HS 2017' : str, 'To SITC Rev. 4':str})
hs_to_sitc['From HS 2017'] = hs_to_sitc['From HS 2017'].apply(lambda x: int(x[:2]))
hs_to_sitc['To SITC Rev. 4'] = hs_to_sitc['To SITC Rev. 4'].apply(lambda x: int(x[:1]))
hs_to_sitc = dict(np.array(hs_to_sitc))

In [55]:
comms = []
years = map(str,range(1999,2018))
for y in years:
    dict_complex = dict(np.array(complexity[['product',y]]))
    temp = copy.copy(commodity)
    temp['year'] = y
    temp['ag1'] = [hs_to_sitc[int(c)] for c in temp['id']]
    temp['name_ag1'] = [sitc_names[c] for c in temp.ag1]

    temp['complexity'] = [dict_complex[int(c)] for c in temp['id']]
    comms.append(temp)

In [56]:
comms_df = pd.concat(comms,ignore_index=1)

In [58]:
countries_dfs = []
years = map(str,range(1999,2018))
for y in years:
    dict_fitness = dict(np.array(fitness[['countries',y]]))
    dict_gdp = dict(np.array(pd.read_csv('ClustersSummaryEU{}.csv'.format(y))[['id','gdp']]))
    dict_gdp_aggr = dict(np.array(pd.read_csv('GDP_aggregate.csv')[['country_code',y]]))
    dict_cluster = dict(np.array(pd.read_csv('ClustersSummaryEU{}.csv'.format(y))[['id','cluster']]))
    temp = copy.copy(countries_df)
    temp['year'] = y
    temp['fitness'] = [dict_fitness[int(c)] for c in temp['id'].apply(lambda x: (dict_code_id[(x)]))]
    temp['gdp'] = [dict_gdp[int(c)] for c in temp['id'].apply(lambda x: (dict_code_id[(x)]))]
    temp['gdp_aggr'] = [dict_gdp_aggr[(c)] for c in temp['id']]
    temp['cluster'] = [dict_cluster[int(c)] for c in temp['id'].apply(lambda x: (dict_code_id[(x)]))]
    try: 
        dict_wages = dict(np.array(pd.read_csv('wages.csv')[['from',y+' ']]))
        temp['wages'] = [dict_wages.get(c,np.NaN) for c in temp['id']]
    except: 
        temp['wages'] = np.NaN
        
        
    try: 
        dict_wages = dict(np.array(pd.read_csv('labour_costs.csv')[['from',y+' ']]))
        temp['labour_costs'] = [dict_wages.get(c,np.NaN) for c in temp['id']]
    except: 
        temp['labour_costs'] = np.NaN
        
    try:
        dict_employ = dict(np.array(pd.read_csv('employment.csv')[['from',y+' ']]))
        temp['employment'] = [dict_employ.get(c,np.NaN) for c in temp['id']]
    except:
        temp['employment'] = np.Nan
        
    try:
        dict_educ = dict(np.array(pd.read_csv('education_secundary.csv')[['from',y+' ']]))
        temp['education'] = [dict_educ.get(c,np.NaN) for c in temp['id']]
    except:
        temp['education'] = np.nan
        
    temp['gdp_aggr'] = temp['gdp_aggr']*1e6



    countries_dfs.append(temp)

In [59]:
country_df = pd.concat(countries_dfs,ignore_index=1)

In [64]:
vertices = pd.concat([comms_df,country_df],ignore_index=1)[['id','name','year','cluster','fitness','gdp','gdp_aggr','labour_costs','wages','employment','education','complexity','ag1','name_ag1']].sort_values(['year','id'])



vertices.to_csv('vertex_data/vertices.csv',index=None)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [63]:
vertices

,id,name,year,cluster,fitness,gdp,gdp_aggr,labour_costs,wages,employment,education,complexity,ag1,name_ag1
0,1,Live animals,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.652587,0.0,Food and live animals
1,2,Meat and edible meat offal,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.886411,0.0,Food and live animals
2,3,"Fish, crustaceans, molluscs, aquatic invertebr...",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121899,0.0,Food and live animals
3,4,"Dairy products, eggs, honey, edible animal pro...",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.787665,0.0,Food and live animals
4,5,"Products of animal origin, nes",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289754,2.0,"Crude materials, inedible, except fuels"
5,6,"Live trees, plants, bulbs, roots, cut flowers etc",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.463841,2.0,"Crude materials, inedible, except fuels"
6,7,Edible vegetables and certain roots and tubers,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.435790,0.0,Food and live animals
7,8,"Edible fruit, nuts, peel of citrus fruit, melons",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.109239,0.0,Food and live animals
8,9,"Coffee, tea, mate and spices",1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032170,0.0,Food and live animals
9,10,Cereals,1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.570500,0.0,Food and live animals


In [16]:
2291697*1e6

2291697000000.0

In [37]:
vertices.ag1.drop_duplicates()

0       0.0
4       2.0
10      5.0
14      4.0
23      1.0
26      3.0
31      8.0
39      6.0
70      9.0
82      7.0
1846    NaN
Name: ag1, dtype: float64

# Arcs

In [31]:
years = list(map(str,range(1999,2018)))
with open('../../data/comtrade/pers_countries.json') as f:
    countries = json.load(f)['results']
countries_id = [c['id'] for c in countries]
countries_name = {c['id']:c['text'] for c in countries}

In [32]:
with open('../../data/comtrade/EU_countries.json') as f:
    countries_EU = json.load(f)['results']
countries_id_EU = [c['id'] for c in countries_EU]
countries_name_EU = {c['id']:c['text'] for c in countries_EU}

In [33]:
files = {y: glob.glob('../../data/comtrade/Export_data_yearly/AG2/{}/Export_{}_*_0.csv'.format(y,y)) for y in years}
files = {y: [f for f in files[y] if f.split('_')[-2] in countries_id] for y in files}

In [34]:
comtrade_df = {y: pd.concat([pd.read_csv(f) for f in files[y]],ignore_index=1) for y in years}

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [35]:
years = map(str,range(1999,2018))
edges = []
for y in years:
    temp = comtrade_df[y][['Year','Reporter Code','Commodity Code','Trade Value (US$)']]
    temp.columns = ['year','from','to','money']
    
    export_product = temp.groupby('to')['money'].sum()
    export_product = dict(zip(export_product.index,export_product))

    export_country = temp.groupby('from')['money'].sum()
    export_country = dict(zip(export_country.index,export_country))

    export_tot = temp['money'].sum()

    temp['exp_prod'] = [export_product[c] for c in temp['to']]
    temp['exp_country'] = [export_country[c] for c in temp['from']]
    temp['exp_tot'] =  export_tot

    temp['rca'] = (temp.money/temp.exp_country) / (temp.exp_prod/temp.exp_tot)
    temp = temp[temp['from'].isin(countries_id_EU)]
    temp['from'] = temp['from'].apply(lambda x: dict_id_code[str(x)])
    #temp = temp[['year','from','to','money','rca']]
    edges.append(temp[['year','from','to','money','rca']])

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [43]:
#temp[temp['from']=='DE'].sort_values('rca',ascending=0)

In [94]:
edges_df = pd.concat(edges,ignore_index=1)
edges_df.to_csv('arc_data/arcs.csv')

In [60]:
edges[-1][(edges[-1]['from']=='IT')&(edges[-1]['to']==87)]

,year,from,to,money,rca
2008,2017,IT,87,43701266259,0.694375


In [94]:
labour = pd.read_csv('lc_lci_lev.tsv',sep='\t')

labour['from'] = labour['unit,lcstruct,nace_r2,geo\\time'].apply(lambda x: x.split(',')[-1])
labour['currency'] = labour['unit,lcstruct,nace_r2,geo\\time'].apply(lambda x: x.split(',')[0])
labour['aggr'] = labour['unit,lcstruct,nace_r2,geo\\time'].apply(lambda x: x.split(',')[1])
labour['code'] = labour['unit,lcstruct,nace_r2,geo\\time'].apply(lambda x: x.split(',')[2])

labour = labour[labour['from'].isin(temp['from'])]
labour = labour[labour['currency']=='EUR']

def convert(x):
    if x==':': return np.NaN
    else: return float(x)

wages = labour[(labour.code=='B')&(labour.aggr=='D11')]
for x in map(str,[2000,2004,2008,2012,2016,2017]):
    x = x+' '
    wages[x] = wages[x].apply(lambda x: x.split(' ')[0])
    wages[x] = wages[x].apply(convert)

    
l_costs = labour[(labour.code=='B')&(labour.aggr=='D12_D4_MD5')]

for x in map(str,[2000,2004,2008,2012,2016,2017]):
    x = x+' '
    l_costs[x] = l_costs[x].apply(lambda x: x.split(' ')[0])
    l_costs[x] = l_costs[x].apply(convert)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/p

In [97]:
l_costs.to_csv('labour_costs.csv',index=None)
wages.to_csv('wages.csv',index=None)

In [36]:
def convert(x):
    if x==':': return np.NaN
    else: return float(x)

employment = pd.read_csv('t2020_10.tsv',sep='\t')
employment['gender'] = employment['indic_em,age,unit,sex,geo\\time'].apply(lambda x: x.split(',')[-2])
employment['from'] = employment['indic_em,age,unit,sex,geo\\time'].apply(lambda x: x.split(',')[-1])
employment = employment[employment['from'].isin(temp['id'])]
employment = employment.query('gender=="T"')
for x in map(str,range(1999,2018)):
    x = x+' '
    employment[x] = employment[x].apply(lambda x: x.split(' ')[0])
    employment[x] = employment[x].apply(convert)

KeyError: 'id'

In [40]:
employment.to_csv('employment.csv',index=None)

In [19]:
labour = pd.read_csv('t2020_10.tsv',sep='\t')

In [16]:
with open('../../data/comtrade/EU_countries.json') as f:
    countries_EU = json.load(f)['results']
countries_id_EU = [c['id'] for c in countries_EU]
countries_name_EU = {c['id']:c['text'] for c in countries_EU}

In [8]:
educ1 = pd.read_csv('../tps00065.tsv',sep='\t')

In [44]:
educ1['sex'] = educ1['sex,age,unit,isced11,geo\\time'].apply(lambda x: x.split(',')[0])
educ1 = educ1[educ1.sex=='T']
educ1['from'] = educ1['sex,age,unit,isced11,geo\\time'].apply(lambda x: x.split(',')[-1])
educ1 = educ1[educ1['from'].isin(temp['from'])]

for x in map(str,range(2007,2018)):
    x = x+' '
    educ1[x] = educ1[x].astype(str)
    educ1[x] = educ1[x].apply(lambda x: x.split(' ')[0])
    educ1[x] = educ1[x].apply(convert)

In [46]:
educ1.to_csv('education_secundary.csv',index=None)